In [1]:
import kagglehub
path = kagglehub.dataset_download("mohankrishnathalla/medical-insurance-cost-prediction")

In [2]:
import pandas as pd
data = pd.read_csv(path + "/medical_insurance.csv")
data.loc[1]
data.dropna()

,person_id,age,sex,region,urban_rural,income,education,marital_status,employment_status,household_size,...,liver_disease,arthritis,mental_health,proc_imaging_count,proc_surgery_count,proc_physio_count,proc_consult_count,proc_lab_count,is_high_risk,had_major_procedure
1,80185,79,Female,North,Urban,12800.0,No HS,Married,Employed,3,...,0,1,1,0,0,1,0,1,1,0
4,92992,53,Male,Central,Suburban,89600.0,Doctorate,Married,Self-employed,2,...,0,1,0,2,0,1,1,0,1,0
5,76435,63,Female,North,Rural,305000.0,HS,Single,Employed,3,...,0,0,0,0,0,0,0,1,1,0
6,84005,36,Male,West,Rural,38900.0,Masters,Single,Employed,1,...,0,0,0,1,0,1,0,1,0,0
7,80918,21,Female,South,Suburban,83700.0,HS,Single,Employed,3,...,0,0,0,2,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99992,87499,53,Male,East,Suburban,73500.0,Masters,Married,Self-employed,2,...,0,0,0,1,0,1,0,0,1,0
99995,6266,50,Male,West,Urban,127200.0,No HS,Married,Employed,2,...,0,0,0,0,0,1,0,0,0,0
99996,54887,42,Male,East,Suburban,21600.0,HS,Married,Employed,2,...,0,0,0,0,0,0,0,0,0,0
99997,76821,41,Male,West,Rural,81900.0,HS,Divorced,Unemployed,1,...,0,0,0,1,0,1,0,0,0,0


In [3]:
data['risk_score'].unique()

array([0.5714, 1.    , 0.2857, 0.8681, 0.6923, 0.1978, 0.3187, 0.8571,
       0.7582, 0.4396, 0.1648, 0.3736, 0.7692, 0.0659, 0.3956, 0.4725,
       0.1538, 0.5494, 0.7473, 0.8352, 0.2747, 0.3846, 0.7143, 0.4615,
       0.9011, 0.5055, 0.5824, 0.4286, 0.3297, 0.978 , 0.3077, 0.7802,
       0.5604, 0.3516, 0.5934, 0.2418, 0.4945, 0.4066, 0.8901, 0.1868,
       0.4505, 0.3626, 0.6703, 0.6264, 0.5385, 0.8242, 0.011 , 0.1319,
       0.4176, 0.2527, 0.3407, 0.0879, 0.2967, 0.1429, 0.6813, 0.4835,
       0.8132, 0.5275, 0.2198, 0.1758, 0.9231, 0.2088, 0.9451, 0.6593,
       0.9341, 0.6484, 0.6044, 0.5165, 0.6374, 0.7363, 0.    , 0.2637,
       0.8022, 0.7253, 0.6154, 0.044 , 0.989 , 0.0989, 0.7033, 0.022 ,
       0.9121, 0.8791, 0.033 , 0.8462, 0.2308, 0.956 , 0.967 , 0.0769,
       0.0549, 0.1099, 0.7912, 0.1209])

In [3]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow import keras
import numpy as np


X = data.drop('risk_score', axis=1)
y = data['risk_score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=86)
categorical_features = ['sex', 'region', 'urban_rural', 'education', 'marital_status', 'employment_status', 'smoker', 'alcohol_freq', 'plan_type', 'network_tier']
numerical_features = [col for col in X.columns if col not in categorical_features]
preprocessor = ColumnTransformer(
	transformers=[
		('num', StandardScaler(), numerical_features),
		('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
	])
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)
input_dim = X_train.shape[1]

model = keras.Sequential()
model.add(keras.layers.InputLayer(input_shape=(input_dim,)))
model.add(keras.layers.Dense(8192, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(4096, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(4096, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(2048, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(1024, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(512, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(256, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(128, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(64, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(32, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(16, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(1))

model.compile(optimizer='adamw', loss='mean_squared_error', metrics=['mean_absolute_error'])

model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.2)

y_pred = model.predict(X_test).flatten()
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"MAE: {mae}, MSE: {mse}, RMSE: {rmse}")
model.save("model.keras", overwrite=True)

2025-12-12 17:14:10.418049: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-12 17:14:10.444886: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-12 17:14:16.372701: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/ubuntu/code_servers/code-server/.venv/lib/python3.12/site-pack

Epoch 1/15


2025-12-12 17:14:23.565577: I external/local_xla/xla/service/service.cc:163] XLA service 0x742d4c002360 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-12 17:14:23.565600: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-12-12 17:14:23.645474: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-12 17:14:24.187042: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002
2025-12-12 17:14:24.298585: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-12-12 17:14:24.298616: I e

  28/2000 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.4090 - mean_absolute_error: 0.4148 

I0000 00:00:1765559683.081784  334433 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2000/2000 ━━━━━━━━━━━━━━━━━━━━ 37s 7ms/step - loss: 0.0058 - mean_absolute_error: 0.0446 - val_loss: 9.7990e-04 - val_mean_absolute_error: 0.0250
Epoch 2/15
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 8.8445e-04 - mean_absolute_error: 0.0231 - val_loss: 8.7211e-04 - val_mean_absolute_error: 0.0233
Epoch 3/15
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 7.8891e-04 - mean_absolute_error: 0.0214 - val_loss: 6.5287e-04 - val_mean_absolute_error: 0.0190
Epoch 4/15
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 5.5665e-04 - mean_absolute_error: 0.0177 - val_loss: 6.7264e-04 - val_mean_absolute_error: 0.0208
Epoch 5/15
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 4.1879e-04 - mean_absolute_error: 0.0150 - val_loss: 3.6793e-04 - val_mean_absolute_error: 0.0131
Epoch 6/15
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 3.9464e-04 - mean_absolute_error: 0.0146 - val_loss: 4.6461e-04 - val_mean_absolute_error: 0.0164
Epoch 7/15
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms

In [7]:
from tensorflow import keras
keras.Sequential().save

<bound method Model.save of <Sequential name=sequential, built=False>>

In [ ]:
json = model.to_json()
with open("model.json", "w") as json_file:
	json_file.write(json)

In [63]:
from pathlib import Path
x = Path('txt/second/third/') / ("asd" + ".txt")
x.parent

PosixPath('txt/second/third')

In [23]:
import tensorflow as tf
from tqdm.notebook import tqdm
import numpy as np
import struct
import ctypes

model = tf.keras.models.load_model("model.keras")
progress = tqdm(total=len(model.layers), desc="Saving weights", unit="layer")
with open ("quant.bin", "wb") as f:
	for layer in model.layers:
		for weight in layer.weights:
			w = weight.numpy()
			if weight.name == 'kernel':
				for i in range(w.shape[0]):
					for j in range(0, w.shape[1], 32):
						w_block = w[i][j:j+32]
						if i == 0 and (j == 8160 or j == 8128):
							print(f'i: {i} j: {j}\n{w_block}')
						scale = (np.max(np.abs(w_block))) / 7
						if scale == 0:
							print("ohno")
						w_block_q = np.clip(np.round(w_block / scale), -7, 7).astype(np.int8)
						w_block_q = w_block_q + 8
						f.write(struct.pack('>f', scale))
						packed = [ctypes.c_uint8((w_block_q[k] & 0x0F) << 4 | (w_block_q[k+1] & 0x0F) if k+1 < len(w_block_q) else 0).value for k in range(0, len(w_block_q), 2)]
						if i == 0 and (j == 8160 or j == 8128):
							print(f'i: {i} j: {j}\n{packed}')
						f.write(bytearray(packed))
			else:
				for i in range(0, w.shape[0], 32):
					w_block = w[i:i+32]
					scale = (np.max(np.abs(w_block))) / 7
					w_block_q = np.clip(np.round(w_block / scale), -7, 7).astype(np.int8)
					w_block_q = w_block_q + 8
					f.write(struct.pack('>f', scale))
					packed = [ctypes.c_uint8((w_block_q[k] & 0x0F) << 4 | (w_block_q[k+1] & 0x0F) if k+1 < len(w_block_q) else 0).value for k in range(0, len(w_block_q), 2)]
					f.write(bytearray(packed))
		progress.update(1)


Saving weights:   0%|          | 0/23 [00:00<?, ?layer/s]

i: 0 j: 8128
[ 0.00656578  0.00551201 -0.01719118 -0.002198   -0.03010065  0.02942467
 -0.01573257  0.00706124  0.00073732  0.00568099  0.00311108 -0.06236957
  0.01257644  0.00053292 -0.00013466  0.00553536  0.01443937 -0.01016005
  0.00269679  0.00015261  0.0052608   0.00184306 -0.00685316  0.00115844
 -0.00643348  0.01115423 -0.00126962  0.00118736  0.00450148 -0.00237069
 -0.00300767  0.00248361]
i: 0 j: 8128
[153, 104, 91, 105, 137, 129, 152, 137, 167, 136, 152, 120, 121, 136, 152, 136]
i: 0 j: 8160
[ 1.7233782e-03  1.7817143e-03  8.5615240e-05 -1.0021025e-03
  9.2019722e-02 -8.7909627e-04 -4.1221022e-03  1.2006492e-03
 -2.5204387e-03 -2.4301734e-02 -4.7882497e-03  5.5040186e-03
 -7.8835413e-03 -1.0640441e-02  4.6611127e-02 -5.3767613e-03
  2.1192238e-03 -2.2610236e-02  7.2979141e-04 -2.0405655e-03
 -3.9625280e-02  6.2561349e-04 -2.8569305e-03  1.4675390e-03
 -1.0176206e-02  2.9928167e-03  7.1384045e-03 -7.6505472e-04
 -2.0220997e-02 -4.7451677e-03 -5.4470249e-03 -4.9842577e-03]
i

In [24]:
import tensorflow as tf
model = tf.keras.models.load_model("model.keras")

In [25]:
for layer in model.layers:
	for weight in layer.weights:
		print(weight.name, weight.shape)

print(len(model.layers))

kernel (83, 8192)
bias (8192,)
gamma (8192,)
beta (8192,)
kernel (8192, 4096)
bias (4096,)
gamma (4096,)
beta (4096,)
kernel (4096, 4096)
bias (4096,)
gamma (4096,)
beta (4096,)
kernel (4096, 2048)
bias (2048,)
gamma (2048,)
beta (2048,)
kernel (2048, 1024)
bias (1024,)
gamma (1024,)
beta (1024,)
kernel (1024, 512)
bias (512,)
gamma (512,)
beta (512,)
kernel (512, 256)
bias (256,)
gamma (256,)
beta (256,)
kernel (256, 128)
bias (128,)
gamma (128,)
beta (128,)
kernel (128, 64)
bias (64,)
gamma (64,)
beta (64,)
kernel (64, 32)
bias (32,)
gamma (32,)
beta (32,)
kernel (32, 16)
bias (16,)
gamma (16,)
beta (16,)
kernel (16, 1)
bias (1,)
23


In [27]:
from IPython.display import display
import json
import numpy as np
import struct
from tqdm.notebook import tqdm
config_data = json.load(open('model.json'))
weights = {}
with open('quant.bin', 'rb') as f:
	bin_data = f.read().hex()
	print(f"Read {len(bin_data)} bytes")
	print(bin_data[:24])
	progress = tqdm(total=len(config_data['config']['layers'])-1, desc="Loading weights", unit="layer", miniters=1)
	ptr = 0
	for layer in config_data['config']['layers']:
		layer_data = []
		if layer['class_name'] == 'InputLayer':
			continue
		if layer['class_name'] == 'Dense':
			d1 = layer['build_config']['input_shape'][1]
			d2 = layer['config']['units']
			w = np.array([])
			batches = d2 // 32
			layer_data = bin_data[ptr:ptr+(((((d1+1)*d2)//32)*8) if d2>=32 else 0)+(8*(d1+1) if d2%32 != 0 else 0)+((d1+1)*(d2+(d2%2)))]
			for i in range(d1):
				w2 = np.array([])
				ptr_2 = (i*batches*40) + (i*(d2-(batches*32)+(d2%2))) + (i*(8 if (batches*32) < d2 else 0))
				if d2 >= 32:
					count1 = 0
					count2 = 0
					for j in range(batches):
						scale_hex = layer_data[ptr_2+(j*40):(ptr_2+(j*40))+8]
						scale = struct.unpack('>f', bytes.fromhex(scale_hex))[0]
						data_hex = layer_data[(ptr_2+(j*40))+8:(ptr_2+(j*40))+40]
						for k in range(0, 16):
							byte = int(data_hex[k*2:(k*2)+2], 16)
							n1 = ((byte >> 4) & 0x0F) - 8
							w2 = np.append(w2, n1 * scale)
							count1 += 1
							n2 = (byte & 0x0F)
							if (n2 != 0):
								w2 = np.append(w2, (n2 - 8) * scale)
								count2 +=1
							if n1 +8 == 0 and n2 == 0:
								print(f'i: {i} j:{j} true')

				if d2 % 32 != 0:
					irreg_val = d2 % 32
					scale_hex = layer_data[ptr_2+(batches*40):ptr_2+(batches*40)+8]
					scale = struct.unpack('>f', bytes.fromhex(scale_hex))[0]
					data_hex = layer_data[ptr_2+(batches*40)+8:ptr_2+(batches*40)+(8+irreg_val)]
					if irreg_val %2 == 1:
						print(f'layer_data: {len(layer_data)}  data_hex: {len(data_hex)}  data_shift: {(batches*40)+(8+irreg_val)}  batches: {batches}  irreg_val: {irreg_val}  d1: {d1}  d2: {d2}')

					for k in range(0, (irreg_val // 2) + (irreg_val % 2)):
						byte = int(data_hex[k*2:(k*2)+2], 16)
						n1 = ((byte >> 4) & 0x0F) - 8
						w2 = np.append(w2, n1 * scale)
						n2 = (byte & 0x0F)
						if (n2 != 0):
							w2 = np.append(w2, (n2 - 8) * scale)
						else:
							print('ohno')
				if len(w) == 0:
					w = w2
				else:
					w = np.vstack((w, w2))

			ptr_2 = (d1*batches*40) + (d1*(d2-(batches*32)+(d2%2))) + (d1*(8 if (batches*32) < d2 else 0))
			w3 = np.array([])
			if (d2 >= 32):
				for i in range(batches):
					scale_hex = layer_data[ptr_2+(i*40):ptr_2+(i*40)+8]
					scale = struct.unpack('>f', bytes.fromhex(scale_hex))[0]
					data_hex = layer_data[ptr_2+(i*40)+8:ptr_2+(i*40)+40]
					for k in range(0, 16):
						byte = int(data_hex[k*2:(k*2)+2], 16)
						n1 = ((byte >> 4) & 0x0F) - 8
						w3 = np.append(w3, n1 * scale)
						n2 = (byte & 0x0F)
						if (n2 != 0):
							w3 = np.append(w3, (n2 - 8) * scale)
			if d2 % 32 != 0:
				irreg_val = d2 % 32
				scale_hex = layer_data[ptr_2+(batches*40):ptr_2+(batches*40)+8]
				scale = struct.unpack('>f', bytes.fromhex(scale_hex))[0]
				data_hex = layer_data[ptr_2+(batches*40)+8:ptr_2+(batches*40)+(8+irreg_val)]
				for k in range(0, (irreg_val // 2) + (irreg_val % 2)):
					byte = int(data_hex[k*2:(k*2)+2], 16)
					n1 = ((byte >> 4) & 0x0F) - 8
					w3 = np.append(w3, n1 * scale)
					n2 = (byte & 0x0F)
					if (n2 != 0):
						w3 = np.append(w3, (n2 - 8) * scale)

			weights[layer['config']['name']] = [w, w3]
					
		if layer['class_name'] == 'LayerNormalization':
			d1 = layer['build_config']['input_shape'][1]
			layer_data = bin_data[ptr:ptr+(((d1//32)*8)+(8 if d1%32 != 0 else 0)+d1+(d1%2))*2]
			w = np.array([])
			batches = d1 // 32
			ptr_2 = 0
			if (d1 >= 32):
				for i in range(d1//32):
					scale_hex = layer_data[i*40:(i*40)+8]
					scale = struct.unpack('>f', bytes.fromhex(scale_hex))[0]
					data_hex = layer_data[(i*40)+8:(i*40)+40]
					for k in range(0, 16):
						byte = int(data_hex[k*2:(k*2)+2], 16)
						n1 = ((byte >> 4) & 0x0F) - 8
						w = np.append(w, n1 * scale)
						n2 = (byte & 0x0F)
						if (n2 != 0):
							w = np.append(w, (n2 - 8) * scale)
			if d1 % 32 != 0:
				irreg_val = d1 % 32
				scale_hex = layer_data[batches*40:(batches*40)+8]
				scale = struct.unpack('>f', bytes.fromhex(scale_hex))[0]
				data_hex = layer_data[(batches*40)+8:(batches*40)+(8+irreg_val)]
				for k in range(0, (irreg_val // 2) + (irreg_val % 2)):
					byte = int(data_hex[k*2:(k*2)+2], 16)
					n1 = ((byte >> 4) & 0x0F) - 8
					w = np.append(w, n1 * scale)
					n2 = (byte & 0x0F)
					if (n2 != 0):
						w = np.append(w, (n2 - 8) * scale)
			ptr_2 = batches*40 + (d1-(batches*32)+(d1%2)) + (8 if batches*32 < d1 else 0)
			w2 = np.array([])
			if (d1 >= 32):
				for i in range(d1//32):
					scale_hex = layer_data[ptr_2+(i*40):ptr_2+(i*40)+8]
					scale = struct.unpack('>f', bytes.fromhex(scale_hex))[0]
					data_hex = layer_data[ptr_2+(i*40)+8:ptr_2+(i*40)+40]
					for k in range(0, 16):
						byte = int(data_hex[k*2:(k*2)+2], 16)
						n1 = ((byte >> 4) & 0x0F) - 8
						w2 = np.append(w2, n1 * scale)
						n2 = (byte & 0x0F)
						if (n2 != 0):
							w2 = np.append(w2, (n2 - 8) * scale)
				ptr_2 += batches*40
			if d1 % 32 != 0:
				irreg_val = d1 % 32
				scale_hex = layer_data[ptr_2+(batches*40):ptr_2+(batches*40)+8]
				scale = struct.unpack('>f', bytes.fromhex(scale_hex))[0]
				data_hex = layer_data[ptr_2+(batches*40)+8:ptr_2+(batches*40)+(8+irreg_val)]
				for k in range(0, (irreg_val // 2) + (irreg_val % 2)):
					byte = int(data_hex[k*2:(k*2)+2], 16)
					n1 = ((byte >> 4) & 0x0F) - 8
					w2 = np.append(w2, n1 * scale)
					n2 = (byte & 0x0F)
					if (n2 != 0):
						w2 = np.append(w2, (n2 - 8) * scale)
			weights[layer['config']['name']] = [w, w2]

		ptr += len(layer_data)
			
		progress.update(1)

Read 77822330 bytes
3d5610f0e188888889888888


Loading weights:   0%|          | 0/23 [00:00<?, ?layer/s]

layer_data: 170  data_hex: 1  data_shift: 9  batches: 0  irreg_val: 1  d1: 16  d2: 1
ohno
layer_data: 170  data_hex: 1  data_shift: 9  batches: 0  irreg_val: 1  d1: 16  d2: 1
ohno
layer_data: 170  data_hex: 1  data_shift: 9  batches: 0  irreg_val: 1  d1: 16  d2: 1
ohno
layer_data: 170  data_hex: 1  data_shift: 9  batches: 0  irreg_val: 1  d1: 16  d2: 1
ohno
layer_data: 170  data_hex: 1  data_shift: 9  batches: 0  irreg_val: 1  d1: 16  d2: 1
ohno
layer_data: 170  data_hex: 1  data_shift: 9  batches: 0  irreg_val: 1  d1: 16  d2: 1
ohno
layer_data: 170  data_hex: 1  data_shift: 9  batches: 0  irreg_val: 1  d1: 16  d2: 1
ohno
layer_data: 170  data_hex: 1  data_shift: 9  batches: 0  irreg_val: 1  d1: 16  d2: 1
ohno
layer_data: 170  data_hex: 1  data_shift: 9  batches: 0  irreg_val: 1  d1: 16  d2: 1
ohno
layer_data: 170  data_hex: 1  data_shift: 9  batches: 0  irreg_val: 1  d1: 16  d2: 1
ohno
layer_data: 170  data_hex: 1  data_shift: 9  batches: 0  irreg_val: 1  d1: 16  d2: 1
ohno
layer_data

In [9]:
from keras.saving import deserialize_keras_object
import json
config_data = json.load(open('model.json'))
model = deserialize_keras_object(config_data)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8192)           │       688,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (None, 8192)           │        16,384 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    33,558,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_1           │ (None, 4096)           │         8,192 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_2           │ (None, 4096)           │         8,192 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2048)           │     8,390,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_3           │ (None, 2048)           │         4,096 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_4           │ (None, 1024)           │         2,048 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_5           │ (None, 512)            │         1,024 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_6           │ (None, 256)            │           512 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_7           │ (None, 128)            │           256 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_8           │ (None, 64)             │           128 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_9           │ (None, 32)             │            64 │
│ (LayerNormalization)            │                        │             

 Total params: 186,772,901 (712.48 MB)

 Trainable params: 62,257,633 (237.49 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 124,515,268 (474.99 MB)

In [11]:
for layer in model.layers:
	name = layer.name
	if layer.name in weights:
		layer.set_weights(weights[name])

NameError: name 'weights' is not defined

In [30]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow import keras
import numpy as np

## Preprocessing
#X = data.drop('risk_score', axis=1)
#y = data['risk_score']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=86)
#categorical_features = ['sex', 'region', 'urban_rural', 'education', 'marital_status', 'employment_status', 'smoker', 'alcohol_freq', 'plan_type', 'network_tier']
#numerical_features = [col for col in X.columns if col not in categorical_features]
#preprocessor = ColumnTransformer(
#	transformers=[
#		('num', StandardScaler(), numerical_features),
#		('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
#	])
#X_train = preprocessor.fit_transform(X_train)
#X_test = preprocessor.transform(X_test)
#input_dim = X_train.shape[1]
# Evaluate the model
y_pred = model.predict(X_test).flatten()
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"MAE: {mae}, MSE: {mse}, RMSE: {rmse}")

625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 932us/step
MAE: 0.8117777000375015, MSE: 0.873895114477833, RMSE: 0.9348235739848632


In [4]:
from uniquant import quantize
quantize("model.keras")

Quantizing weights:   0%|          | 0/23 [00:00<?, ?layer/s]

Quantizing done. Quant saved to: model.uniq


In [6]:
from uniquant import dequantize_save

model = dequantize_save("model.uniq", model_name='dequant')

Dequantizing weights:   0%|          | 0/23 [00:00<?, ?layer/s]

Dequantizing done. Model returned from function. Also model saved to: dequant.keras


In [ ]:
results = []

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow import keras
import numpy as np
from uniquant import quantize, dequantize
from tqdm.auto import tqdm
keras.Sequential().save
# Preprocessing
X = data.drop('risk_score', axis=1)
y = data['risk_score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=86)
categorical_features = ['sex', 'region', 'urban_rural', 'education', 'marital_status', 'employment_status', 'smoker', 'alcohol_freq', 'plan_type', 'network_tier']
numerical_features = [col for col in X.columns if col not in categorical_features]
preprocessor = ColumnTransformer(
	transformers=[
		('num', StandardScaler(), numerical_features),
		('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
	])
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)
input_dim = X_train.shape[1]
# Build the model
model = keras.Sequential()
model.add(keras.layers.InputLayer(input_shape=(input_dim,)))
model.add(keras.layers.Dense(8192, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(4096, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(4096, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(2048, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(1024, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(512, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(256, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(128, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(64, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(32, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(16, activation='gelu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(1))
# Compile the model
model.compile(optimizer='adamw', loss='mean_squared_error', metrics=['mean_absolute_error'])
# Train the model
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.2)
# Evaluate the model
y_pred = model.predict(X_test).flatten()
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"MAE: {mae}, MSE: {mse}, RMSE: {rmse}")
results.append({"mae": mae, "mse": mse, "rmse": rmse})
model.save("model.keras", overwrite=True)
del(model)
del(X_train)
del(y_train)

progress = tqdm(total=5, desc="Tests", unit="test", miniters=1, mininterval=0)
for num in [32,64,128,16,8]:
	quantize("model.keras", overwrite=True)
	model = dequantize("model.uniq")
	y_pred = model.predict(X_test).flatten()
	mae = mean_absolute_error(y_test, y_pred)
	mse = mean_squared_error(y_test, y_pred)
	rmse = np.sqrt(mse)
	print(f"MAE: {mae}, MSE: {mse}, RMSE: {rmse}")
	results.append({"mae": mae, "mse": mse, "rmse": rmse})
	del model
	progess.update(1)


In [ ]:
from uniquant import quantize
quantize("model.keras", overwrite=True, pack_size=4)

2025-12-12 20:08:55.281491: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-12 20:08:55.319579: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-12 20:08:56.087348: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1765570136.870309  405475 gpu_device.cc:2020] Created de

Quantizing weights:   0%|          | 0/23 [00:00<?, ?layer/s]